In [1]:
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request
from quantumnet.objects import Request
rede = Network()
controlador = Controller(rede)
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Criação de requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 5
print("Hosts:", len(rede.hosts))    
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))
requestscopy = requests.copy()
print("Requisições:", requests)

Hosts inicializados
Canais inicializados
Pares EPRs adicionados
Hosts: 12
Requisições: [<quantumnet.objects.request.Request object at 0x0000020688D582F0>, <quantumnet.objects.request.Request object at 0x0000020688856490>, <quantumnet.objects.request.Request object at 0x00000206888565D0>, <quantumnet.objects.request.Request object at 0x000002068884AC40>, <quantumnet.objects.request.Request object at 0x000002068884AD70>]


In [2]:
# Parte adicional para simulação com timeslot
def simulate_requests_with_timeslots(rede, requests, controlador):
    """
    Simula o processamento de requests com a introdução de timeslots.
    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.
    """
    current_time = 0  # Tempo global da simulação
    for request in requests:
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = current_time
            current_time += 3  # Delay de 3 time-slots para registrar nova regra
            controlador.add_match_route_rule_in_host_reactive(request)
            rule = alice.find_rule_by_request(request)
            controlador.run_rule(rule[1])
            request.endtime = current_time
        else:  # Caso já exista a regra
            request.starttime = current_time
            current_time += 1  # Delay de 1 time-slot para execução
            controlador.run_rule(rule[1])
            request.endtime = current_time
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

In [3]:
simulate_requests_with_timeslots(rede, requests, controlador)

Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
    Ação: Swap(10-6-2)
Running Swap(10-6-2)
Request R-D582F0: Start Time = 0, End Time = 3
Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
    Ação: Swap(9-5-6)
Running Swap(9-5-6)
Request R-856490: Start Time = 3, End Time = 6
Decisão aplicada: (Normal E2E,)
Passo: 1
Request R-8565D0: Start Time = 6, End Time = 9
Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
    Ação: Swap(10-2-1)
Running Swap(10-2-1)
Request R-84AC40: Start Time = 9, End Time = 12
Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
    Ação: Swap(3-2-6)
Running Swap(3-2-6)
Request R-84AD70: Start Time = 12, End Time = 15


In [4]:
# Parte adicional para simulação com timeslot
def simulate_requests_with_timeslots(rede, requests, controlador):
    """
    Simula o processamento de requests com timeslots contínuos.
    - O tempo passa de 1 em 1 independentemente da requisição.
    - Nova regra leva 3 time-slots para ser registrada.
    - As regras são executadas no momento correto assim que o tempo alcança o necessário.
    """
    current_time = 0  # Tempo global da simulação
    pending_requests = []  # Lista de requisições pendentes para registro
    
    while requests or pending_requests:
        # Log do tempo atual
        print(f"Time Slot: {current_time}")
        # Processar requests pendentes para execução
        for pending in pending_requests[:]:
            if pending['start_time'] + 3 == current_time:  # Após 3 time-slots
                controlador.add_match_route_rule_in_host_reactive(pending['request'])
                pending['request'].endtime = current_time
                print(f"Request {pending['request']}: Start Time = {pending['request'].starttime}, End Time = {pending['request'].endtime}")
                pending_requests.remove(pending)
        
        # Processar nova requisição apenas se ainda há requests
        if requests:
            request = requests.pop(0)
            alice = rede.get_host(request.alice)
            rule = alice.find_rule_by_request(request)
            
            if rule == False:  # Caso não exista um match na tabela
                request.starttime = current_time
                pending_requests.append({'request': request, 'start_time': current_time})
            else:  # Caso já exista a regra
                request.starttime = current_time
                controlador.run_rule(rule[1])
                request.endtime = current_time + 1
                print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")
        
        # Incrementar o tempo global
        current_time += 1
    
simulate_requests_with_timeslots(rede, requests, controlador)

Time Slot: 0
Passo: 1
Passo: 2
    Ação: Swap(10-6-2)
Running Swap(10-6-2)
Request R-D582F0: Start Time = 0, End Time = 1
Time Slot: 1
Passo: 1
Passo: 2
    Ação: Swap(9-5-6)
Running Swap(9-5-6)
Request R-856490: Start Time = 1, End Time = 2
Time Slot: 2
Passo: 1
Request R-8565D0: Start Time = 2, End Time = 3
Time Slot: 3
Passo: 1
Passo: 2
    Ação: Swap(10-2-1)
Running Swap(10-2-1)
Request R-84AC40: Start Time = 3, End Time = 4
Time Slot: 4
Passo: 1
Passo: 2
    Ação: Swap(3-2-6)
Running Swap(3-2-6)
Request R-84AD70: Start Time = 4, End Time = 5


In [5]:
simulate_requests_with_timeslots(rede, requestscopy, controlador)

Time Slot: 0
Passo: 1
Passo: 2
    Ação: Swap(10-6-2)
Running Swap(10-6-2)
Request R-D582F0: Start Time = 0, End Time = 1
Time Slot: 1
Passo: 1
Passo: 2
    Ação: Swap(9-5-6)
Running Swap(9-5-6)
Request R-856490: Start Time = 1, End Time = 2
Time Slot: 2
Passo: 1
Request R-8565D0: Start Time = 2, End Time = 3
Time Slot: 3
Passo: 1
Passo: 2
    Ação: Swap(10-2-1)
Running Swap(10-2-1)
Request R-84AC40: Start Time = 3, End Time = 4
Time Slot: 4
Passo: 1
Passo: 2
    Ação: Swap(3-2-6)
Running Swap(3-2-6)
Request R-84AD70: Start Time = 4, End Time = 5
